In [7]:
from matplotlib import pyplot as plt
%matplotlib inline

In [8]:
from __future__ import print_function

import os
import subprocess

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [9]:
file = '20160301_TE_survey_cleaned.csv'

In [10]:
df = pd.read_csv(file)

In [36]:
df.columns.values

array(['ID', 'T(K)', 'Z*10^-4 reported', 'Resist(Ohm.cm)',
       'Seebeck(uV/K)', 'kappaZT', 'Unnamed: 6', 'Resist(400K)',
       'Seebeck(400K)', 'Pf(W/K^2/m)', 'ZT', 'kappa(W/mK)', 'x',
       'Formula', 'series', 'T_Max', 'family', 'Unnamed: 17',
       'Unnamed: 18', 'Conduct(S/cm)', 'Power_Factor*T_(W/mK)',
       'preparative_route', 'final_form', 'cell_volume(A^3)',
       'formula_units_per_cell', 'atoms_per_formula_unit',
       'total_atoms_per_unit_cell', 'average_atomic_volume',
       'ICSD_of_structure', 'temp of ICSD (K)', 'S^2', 'ke/ktotal',
       'space_group', 'number_symmetry_elements', 'Unnamed: 34',
       'Unnamed: 35', 'bin'], dtype=object)

In [5]:
df['Seebeck(uV/K)'].describe()

count    1098.000000
mean      -40.155270
std       192.691410
min      -752.196000
25%      -163.359500
50%       -67.600000
75%        99.358325
max      1235.430000
Name: Seebeck(uV/K), dtype: float64

### For the decision tree we need to bin the target values, so that we create a mock classification problem.

In [6]:
df['bin'] = pd.cut(df['Seebeck(uV/K)'], 50) #Breaking the Seebeck column into 50
                                            #bins, this number is arbitraty.

In [7]:
df = df.sort_values(['bin'], ascending = True) #sort the dataframe by ascending order
                                                #of the bins.

In [8]:
df['bin'].head()

1098                     NaN
1031    (-754.184, -712.443]
328     (-672.691, -632.938]
1032    (-632.938, -593.186]
124     (-632.938, -593.186]
Name: bin, dtype: category
Categories (50, interval[float64]): [(-754.184, -712.443] < (-712.443, -672.691] < (-672.691, -632.938] < (-632.938, -593.186] ... (1076.42, 1116.172] < (1116.172, 1155.925] < (1155.925, 1195.677] < (1195.677, 1235.43]]

In [9]:
#Pulling the columns we're interested in using for our predictors.
reduced_df = df[['Resist(Ohm.cm)', 'T(K)', 'Seebeck(uV/K)', 'average_atomic_volume', 'space_group', 'bin']]

In [10]:
reduced_df.head()

,Resist(Ohm.cm),T(K),Seebeck(uV/K),average_atomic_volume,space_group,bin
1098,NaN,NaN,NaN,NaN,NaN,NaN
1031,2.918890,300.0,-752.196,13.5200,139.0,"(-754.184, -712.443]"
328,0.302535,300.0,-650.910,10.3485,62.0,"(-672.691, -632.938]"
1032,2.161640,400.0,-618.425,13.5200,139.0,"(-632.938, -593.186]"
124,0.460937,400.0,-600.164,10.3485,62.0,"(-632.938, -593.186]"


### I need to assign each bin a number.  I created a new column with these bin numbers.

In [11]:
def encode_target(df, target_column):
    """Add column to data that assigns integers to the binned Seebeck data"""
    df_mod = df.copy()
    targets = df_mod[target_column].unique()
    map_to_int = {name: n for n, name in enumerate(targets)}
    df_mod['Target'] = df_mod[target_column].replace(map_to_int)
    
    return (df_mod, targets)

In [12]:
df2, targets = encode_target(reduced_df, 'bin')

In [13]:
df2.head()

,Resist(Ohm.cm),T(K),Seebeck(uV/K),average_atomic_volume,space_group,bin,Target
1098,NaN,NaN,NaN,NaN,NaN,NaN,0
1031,2.918890,300.0,-752.196,13.5200,139.0,"(-754.184, -712.443]",1
328,0.302535,300.0,-650.910,10.3485,62.0,"(-672.691, -632.938]",2
1032,2.161640,400.0,-618.425,13.5200,139.0,"(-632.938, -593.186]",3
124,0.460937,400.0,-600.164,10.3485,62.0,"(-632.938, -593.186]",3


In [14]:
#Clean up the NA values
df3 = df2.dropna()

In [124]:
#Break data into x and y
df3_x = pd.DataFrame(df3[['Resist(Ohm.cm)', 'T(K)', 'average_atomic_volume', 'space_group']])
df3_y = pd.DataFrame(df3['Target'])
df3_reg_y = pd.DataFrame(df3['Seebeck(uV/K)'])

In [16]:
#this just makes it easy to call all x parameters later on.
features = list(df3_x.columns[:4])

In [83]:
print (features)

['Resist(Ohm.cm)', 'T(K)', 'average_atomic_volume', 'space_group']


In [84]:
#Split into test/train
x_train, x_test = train_test_split(df3_x, test_size=0.2)
y_train, y_test = train_test_split(df3_y, test_size=0.2)

In [121]:
#Fit the decision tree,
#1st iteration dt = DecisionTreeRegressor(min_samples_split = 20, random_state = 99)
#Predictor gets better when max depth is decreased, but you loose too much info after a certain point.
y_fit_cla = y_train['Target']
X_fit_cla = x_train[features]
dt_cla = DecisionTreeClassifier(max_depth = 10, random_state = 99)
dt_fit_cla = dt_cla.fit(X_fit_cla, y_fit_cla)

In [122]:
dt_cla_pred = dt_cla.predict(x_test[features])
score = accuracy_score(y_test, dt_pred)
print(score)

0.07009345794392523


In [125]:
#Split into test/train
x_train_reg, x_test_reg = train_test_split(df3_x, test_size=0.2)
y_train_reg, y_test_reg = train_test_split(df3_reg_y, test_size=0.2)

In [170]:
#Fit the decision tree,
#1st iteration dt = DecisionTreeRegressor(min_samples_split = 20, random_state = 99)
#Predictor gets better when max depth is decreased, but you loose too much info after a certain point.
y_fit = y_train_reg['Seebeck(uV/K)']
X_fit = x_train[features]
dt = DecisionTreeRegressor(max_depth = 10, random_state = 99)
dt_fit = dt.fit(x_train_reg, y_train_reg)

In [171]:
#Check the predictor.
dt_pred = dt.predict(x_test[features])
score = dt.score(x_test_reg, y_test_reg)
print (score)

-0.6167586431370182


In [134]:
#Random Forest is a different call, lets  try this out.
rdt = RandomForestRegressor(random_state=1010, n_estimators=100, max_depth = 10)
rdt_fit = rdt.fit(x_train_reg[features], y_train_reg['Seebeck(uV/K)'])
score = rdt.score(x_test_reg, y_test_reg)
print (score)

-0.14706159007854258


In [169]:
#Random Forest is a different call, lets  try this out.
rdt = RandomForestClassifier(random_state=1010, n_estimators=100, max_depth = 10)
rdt_fit = rdt.fit(x_train[features], y_train['Target'])
score = rdt.score(x_test, y_test)
print (score)

0.0514018691588785


In [20]:
#I was following along with a demo and the author mentioned a way to visalize
#the tree.  I haven't been able to get this to work yet.
def visualize_tree(tree, feature_names):
    """Create tree png using graphviz.

    Args
    ----
    tree -- scikit-learn DecsisionTree.
    feature_names -- list of feature names.
    """
    with open("dt.dot", 'w') as f:
        export_graphviz(tree, out_file=f,
                        feature_names=feature_names)

    command = ["dot", "-Tpng", "dt.dot", "-o", "dt.png"]
    try:
        subprocess.check_call(command)
    except:
        exit("Could not run dot, ie graphviz, to "
             "produce visualization")

In [21]:
visualize_tree(dt, features)

In [51]:
def encode_columns(df, df_mod2, target_column):
    """Change unique column values to integers"""
    targets = df_mod2[target_column].unique()
    map_to_int = {name: n for n, name in enumerate(targets)}
    df_mod2[target_column] = df_mod2[target_column].replace(map_to_int)
    
    return

In [138]:
df_mod2 = df.copy()
encode_columns(df, df_mod2, 'preparative_route')

In [139]:
df_mod2['preparative_route'].head()

1098    0
1031    1
328     2
1032    3
124     2
Name: preparative_route, dtype: int64

In [140]:
reduced_df2 = df_mod2[['T(K)', 'Resist(Ohm.cm)', 'Seebeck(uV/K)', 'Resist(400K)', 'Conduct(S/cm)', 'preparative_route', 
                    'cell_volume(A^3)', 'average_atomic_volume', 'space_group', 'number_symmetry_elements', 'bin']]

In [141]:
reduced_df3 = reduced_df2.dropna()

In [142]:
df4, targets = encode_target(reduced_df3, 'bin')

In [159]:
df4_x_1 = pd.DataFrame(df4[['T(K)', 'Resist(Ohm.cm)', 'Resist(400K)', 'Conduct(S/cm)', 'preparative_route', 
                    'cell_volume(A^3)', 'average_atomic_volume', 'space_group', 'number_symmetry_elements']])
df4_y_1 = pd.DataFrame(df4['Target'])
df4_reg_y_1 = pd.DataFrame(df4['Seebeck(uV/K)'])

In [162]:
x_train_1, x_test_1 = train_test_split(df4_x_1, test_size=0.2)
y_train_1, y_test_1 = train_test_split(df4_y_1, test_size=0.2)
y_train_reg_1, y_train_reg_2 = train_test_split(df4_reg_y_1, test_size=0.2)

In [152]:
features_1 = list(df4_x_1.columns[:9])

In [153]:
print (features_1)

['T(K)', 'Resist(Ohm.cm)', 'Resist(400K)', 'Conduct(S/cm)', 'preparative_route', 'cell_volume(A^3)', 'average_atomic_volume', 'space_group', 'number_symmetry_elements']


In [154]:
dt_1 = DecisionTreeClassifier(max_depth = 10, random_state = 99)
dt_fit_1 = dt_1.fit(x_train_1[features_1], y_train_1['Target'],)

In [156]:
dt_pred_1 = dt_fit_1.score(x_test_1, y_test_1)
print (dt_pred_1)

0.014705882352941176


In [165]:
rdt_1 = RandomForestRegressor(random_state=1010, n_estimators=100, max_depth = 10)
rdt_fit_1 = rdt_1.fit(x_train_1[features_1], y_train_reg_1['Seebeck(uV/K)'])

In [166]:
rdt_pred_1 = rdt_fit_1.score(x_test_1, y_train_reg_2)
print (rdt_pred)

-0.1205386828809325


In [2]:
import plotly
plotly.__version__

'2.4.1'

In [3]:
import plotly.plotly as py
import plotly.figure_factory as ff

In [1]:
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.sampledata.periodic_table import elements
from bokeh.transform import dodge, factor_cmap

output_file("periodic.html")

periods = ["I", "II", "III", "IV", "V", "VI", "VII"]
groups = [str(x) for x in range(1, 19)]

df = elements.copy()
df["atomic mass"] = df["atomic mass"].astype(str) 
df["group"] = df["group"].astype(str)
df["period"] = [periods[x-1] for x in df.period]
df = df[df.group != "-"] #cleaning elements dataframe?
df = df[df.symbol != "Lr"]
df = df[df.symbol != "Lu"]

cmap = {
    "alkali metal"         : "#a6cee3",
    "alkaline earth metal" : "#1f78b4",
    "metal"                : "#d93b43",
    "halogen"              : "#999d9a",
    "metalloid"            : "#e08d49",
    "noble gas"            : "#eaeaea",
    "nonmetal"             : "#f1d4Af",
    "transition metal"     : "#599d7A",
} # Creating a dictionary?

source = ColumnDataSource(df)

p = figure(title="Periodic Table (omitting LA and AC Series)", plot_width=1000, plot_height=450,
           tools="", toolbar_location=None,
           x_range=groups, y_range=list(reversed(periods)))

p.rect("group", "period", 0.95, 0.95, source=source, fill_alpha=0.6, legend="metal",
       color=factor_cmap('metal', palette=list(cmap.values()), factors=list(cmap.keys())))

text_props = {"source": source, "text_align": "left", "text_baseline": "middle"}

x = dodge("group", -0.4, range=p.x_range)

r = p.text(x=x, y="period", text="symbol", **text_props)
r.glyph.text_font_style="bold"

r = p.text(x=x, y=dodge("period", 0.3, range=p.y_range), text="atomic number", **text_props)
r.glyph.text_font_size="8pt"

r = p.text(x=x, y=dodge("period", -0.35, range=p.y_range), text="name", **text_props)
r.glyph.text_font_size="5pt"

r = p.text(x=x, y=dodge("period", -0.2, range=p.y_range), text="atomic mass", **text_props)
r.glyph.text_font_size="5pt"

p.text(x=["3", "3"], y=["VI", "VII"], text=["LA", "AC"], text_align="center", text_baseline="middle")

p.add_tools(HoverTool(tooltips = [
    ("Name", "@name"),
    ("Atomic number", "@{atomic number}"),
    ("Atomic mass", "@{atomic mass}"),
    ("Type", "@metal"),
    ("CPK color", "$color[hex, swatch]:CPK"),
    ("Electronic configuration", "@{electronic configuration}"),
]))

p.outline_line_color = None
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_standoff = 0
p.legend.orientation = "horizontal"
p.legend.location ="top_center"

show(p)

In [3]:
from temann.interpret import get_empirical_formula

In [17]:
periodic_df = df[['Seebeck(uV/K)', 'Pf(W/K^2/m)', 'Formula', 'T(K)']]

In [22]:
periodic_df.dropna()

,Seebeck(uV/K),Pf(W/K^2/m),Formula,T(K)
0,-462.967000,4.286770e-07,CaMnO3,300.0
1,-462.967000,4.286770e-07,CaMnO3,400.0
2,-470.651000,9.162900e-06,CaMnO3,700.0
3,-279.829000,4.411760e-05,CaMnO3,1000.0
4,-201.708000,8.822830e-05,Ca0.98La0.02MnO3,300.0
5,-201.708000,8.822830e-05,Ca0.98La0.02MnO3,400.0
6,-224.120000,7.721820e-05,Ca0.98La0.02MnO3,700.0
7,-190.181000,8.803530e-05,Ca0.98La0.02MnO3,1000.0
8,-123.586000,1.001180e-04,Ca0.96La0.04MnO3,300.0
9,-123.586000,1.001180e-04,Ca0.96La0.04MnO3,400.0


In [31]:
periodic_df1 = periodic_df[periodic_df['Seebeck(uV/K)'] < -150] 

In [32]:
periodic_df1.append(periodic_df[periodic_df['Seebeck(uV/K)'] > 150])

,Seebeck(uV/K),Pf(W/K^2/m),Formula,T(K)
0,-462.967,4.286770e-07,CaMnO3,300.0
1,-462.967,4.286770e-07,CaMnO3,400.0
2,-470.651,9.162900e-06,CaMnO3,700.0
3,-279.829,4.411760e-05,CaMnO3,1000.0
4,-201.708,8.822830e-05,Ca0.98La0.02MnO3,300.0
5,-201.708,8.822830e-05,Ca0.98La0.02MnO3,400.0
6,-224.120,7.721820e-05,Ca0.98La0.02MnO3,700.0
7,-190.181,8.803530e-05,Ca0.98La0.02MnO3,1000.0
10,-173.533,1.389120e-04,Ca0.96La0.04MnO3,700.0
11,-160.085,1.153540e-04,Ca0.96La0.04MnO3,1000.0


In [36]:
periodic_df1.rename(columns = {'T(K)' : 'T_K'})

,Seebeck(uV/K),Pf(W/K^2/m),Formula,T_K
0,-462.967,4.286770e-07,CaMnO3,300.0
1,-462.967,4.286770e-07,CaMnO3,400.0
2,-470.651,9.162900e-06,CaMnO3,700.0
3,-279.829,4.411760e-05,CaMnO3,1000.0
4,-201.708,8.822830e-05,Ca0.98La0.02MnO3,300.0
5,-201.708,8.822830e-05,Ca0.98La0.02MnO3,400.0
6,-224.120,7.721820e-05,Ca0.98La0.02MnO3,700.0
7,-190.181,8.803530e-05,Ca0.98La0.02MnO3,1000.0
10,-173.533,1.389120e-04,Ca0.96La0.04MnO3,700.0
11,-160.085,1.153540e-04,Ca0.96La0.04MnO3,1000.0


In [39]:
periodic_df1['T(K)'].unique()

array([ 300.,  400.,  700., 1000.,   nan])

In [ ]:
periodic_df1 = periodic_df[periodic_df['Seebeck(uV/K)'] < -150] 

In [46]:
T_300 = pd.DataFrame(periodic_df1[periodic_df1['T(K)'] == 300])

In [49]:
T_400 = pd.DataFrame(periodic_df1[periodic_df1['T(K)'] == 400])

In [50]:
T_700 = pd.DataFrame(periodic_df1[periodic_df1['T(K)'] == 700])

In [51]:
T_1000 = pd.DataFrame(periodic_df1[periodic_df1['T(K)'] == 1000])

In [67]:
def parse_multiple_formulas(df, target_df):
    for idx in range(len(df['Formula'])):
        parsed_formula = get_empirical_formula(df['Formula'].iloc[idx])
        target_df.append(parsed_formula)
        idx = idx + 1
    return (target_df)

In [68]:
parsed_T_300 = []
parse_multiple_formulas(periodic_df1, parsed_T_300)

[{'Ca': 1, 'Mn': 1, 'O': 3},
 {'Ca': 1, 'Mn': 1, 'O': 3},
 {'Ca': 1, 'Mn': 1, 'O': 3},
 {'Ca': 1, 'Mn': 1, 'O': 3},
 {'Ca': 0.98, 'La': 0.02, 'Mn': 1, 'O': 3},
 {'Ca': 0.98, 'La': 0.02, 'Mn': 1, 'O': 3},
 {'Ca': 0.98, 'La': 0.02, 'Mn': 1, 'O': 3},
 {'Ca': 0.98, 'La': 0.02, 'Mn': 1, 'O': 3},
 {'Ca': 0.96, 'La': 0.04, 'Mn': 1, 'O': 3},
 {'Ca': 0.96, 'La': 0.04, 'Mn': 1, 'O': 3},
 {'Ca': 1, 'Mn': 1, 'O': 3},
 {'Ca': 1, 'Mn': 1, 'O': 3},
 {'Ca': 1, 'Mn': 1, 'O': 3},
 {'Ca': 1, 'Mn': 1, 'O': 3},
 {'Bi': 0.02, 'Ca': 0.98, 'Mn': 0.98, 'Nb': 0.02, 'O': 3},
 {'Bi': 0.02, 'Ca': 0.98, 'Mn': 0.98, 'Nb': 0.02, 'O': 3},
 {'Bi': 0.04, 'Ca': 0.96, 'Mn': 0.96, 'Nb': 0.04, 'O': 3},
 {'Bi': 0.1, 'Ca': 0.9, 'Mn': 0.9, 'Nb': 0.1, 'O': 3},
 {'Ca': 1, 'Mn': 0.98, 'Nb': 0.02, 'O': 3},
 {'Ca': 1, 'Mn': 0.98, 'Nb': 0.02, 'O': 3},
 {'Ca': 1, 'Mn': 0.98, 'Nb': 0.02, 'O': 3},
 {'Ca': 1, 'Mn': 0.98, 'Nb': 0.02, 'O': 3},
 {'Ca': 1, 'Mn': 0.98, 'O': 3, 'Ru': 0.02},
 {'Ca': 1, 'Mn': 0.98, 'O': 3, 'Ru': 0.02},
 {'Ca': 

In [69]:
parsed_T_300_df = pd.DataFrame(parsed_T_300)

In [70]:
parsed_T_300_df.head()

,Al,Ba,Bi,Ca,Co,Cu,Dy,Fe,Ga,Gd,...,Sn,Sr,Tb,Te,Ti,Tl,Y,Yb,Zn,Zr
0,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,0.98,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df1[df1.notnull()].count()

In [84]:
results_T_300 = parsed_T_300_df[parsed_T_300_df.notnull()].count()

In [85]:
print (results_T_300)

Al     41
Ba     13
Bi     17
Ca     62
Co     40
Cu      7
Dy      1
Fe     29
Ga      8
Gd      8
Ge     12
Hf     33
In     30
K       2
La     20
Li      1
Mg     15
Mn     62
Nb     42
Nd      7
Ni     67
O     175
P       4
Pb      3
Ru      4
S       1
Sb     29
Se      2
Si     27
Sm      2
Sn     81
Sr     24
Tb      1
Te      6
Ti     71
Tl      1
Y       8
Yb      2
Zn     55
Zr     44
dtype: int64


In [86]:
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.sampledata.periodic_table import elements
from bokeh.transform import dodge, factor_cmap

output_file("periodic.html")

periods = ["I", "II", "III", "IV", "V", "VI", "VII"]
groups = [str(x) for x in range(1, 19)]

df = elements.copy()
df["atomic mass"] = df["atomic mass"].astype(str) 
df["group"] = df["group"].astype(str)
df["period"] = [periods[x-1] for x in df.period]
df = df[df.group != "-"] #cleaning elements dataframe?
df = df[df.symbol != "Lr"]
df = df[df.symbol != "Lu"]

cmap = {
    "alkali metal"         : "#a6cee3",
    "alkaline earth metal" : "#1f78b4",
    "metal"                : "#d93b43",
    "halogen"              : "#999d9a",
    "metalloid"            : "#e08d49",
    "noble gas"            : "#eaeaea",
    "nonmetal"             : "#f1d4Af",
    "transition metal"     : "#599d7A",
} # Creating a dictionary?

source = ColumnDataSource(df)

p = figure(title="Periodic Table (omitting LA and AC Series)", plot_width=1000, plot_height=450,
           tools="", toolbar_location=None,
           x_range=groups, y_range=list(reversed(periods)))

p.rect("group", "period", 0.95, 0.95, source=source, fill_alpha=0.6, legend="metal",
       color=factor_cmap('metal', palette=list(cmap.values()), factors=list(cmap.keys())))

text_props = {"source": source, "text_align": "left", "text_baseline": "middle"}

x = dodge("group", -0.4, range=p.x_range)

r = p.text(x=x, y="period", text="symbol", **text_props)
r.glyph.text_font_style="bold"

r = p.text(x=x, y=dodge("period", 0.3, range=p.y_range), text="atomic number", **text_props)
r.glyph.text_font_size="8pt"

r = p.text(x=x, y=dodge("period", -0.35, range=p.y_range), text="name", **text_props)
r.glyph.text_font_size="5pt"

r = p.text(x=x, y=dodge("period", -0.2, range=p.y_range), text="atomic mass", **text_props)
r.glyph.text_font_size="5pt"

p.text(x=["3", "3"], y=["VI", "VII"], text=["LA", "AC"], text_align="center", text_baseline="middle")

p.add_tools(HoverTool(tooltips = [
    ("Name", "@name"),
    ("Atomic number", "@{atomic number}"),
    ("Atomic mass", "@{atomic mass}"),
    ("Type", "@metal"),
    ("CPK color", "$color[hex, swatch]:CPK"),
    ("Electronic configuration", "@{electronic configuration}"),
]))

p.outline_line_color = None
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_standoff = 0
p.legend.orientation = "horizontal"
p.legend.location ="top_center"

show(p)